## PROTEUS2 protein secondary structure analysis:

### This is a notebook to organize and evaluate the output of protein secondary structure prediction by [Proteus2](http://www.proteus2.ca/proteus2/), described in [Montgomerie et al., 2008](https://academic.oup.com/nar/article/36/suppl_2/W202/2506231) in _Nucleic Acids Research_. 

### The output of Proteus2 comes in an email in nominally FASTA format, but with extra line breaks and spacings within protein and prediction sequences. Also, the name gets cut off.

#### They look like this:

#### >Thalas

MMKLAALAAL MGSAAAFAPA QTGKASTQLR AFEDELGAQP PLGFFDPFGM 

CCHHHHHHHH HHHHHCCCCC CCCCCCCCCC CCCCCCCCCC CCCCCCCCCC 

LSGDCTQERF DRLRYVEIKH GRICMLAFLG QIVTRAGIHL PGSINYAGDS 

CCCCCCHHHH HHHHHHHHHH HHHHHHHHHH HHHHHHHCCC CCCCCCCCCC 

FDSFPNGVAA LFGPNSIPTA GLVQIIAFIG VLECAFMRDV PGTGNEFVGD 

CCCCCCCCCC CCCCCCCCHH HHHHHHHHHH HHHHHHHHCC CCCCCCCCCC 

FRNGYIDFGW DDFDEETKLQ KRAIQSGTIS NMMKLAALAA LMGSAAAFAP 

CCCCCCCCCC CCCCHHHHHH HHHHHHHHHH HHHHHHHHHC CCCCCCCCCC 



### Output means:

- H = Helix
- E = Beta Strand
- C = Coil
- T = Membrane helix
- B = Membrane strand
- S = Signal peptide
- c = Cleavage site

In [1]:
cd /home/millieginty/Documents/git-repos/rot-mayer/analyses/proteus2/Proteins-Proteus2/

In [2]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [3]:
ls

PeaksDB_322_prot_trypsin_totals
PeaksDB_323_prot_trypsin_totals
PeaksDB_324_prot_trypsin_totals
PeaksDB_325_prot_trypsin_totals
PeaksDB_329_prot_undigested_totals
PeaksDB_330_prot_undigested_totals
T0_322_trypsin_PeaksDB_protein_proteus
T0_322_trypsin_PeaksDB_protein_proteus_sort.csv
T0_322_trypsin_PeaksDB_protein_proteus.txt
T0_329_undigested_PeaksDB_protein_proteus
T0_329_undigested_PeaksDB_protein_proteus_sort.csv
T0_329_undigested_PeaksDB_protein_proteus.txt
T12_325_trypsin_PeaksDB_protein_proteus
T12_325_trypsin_PeaksDB_protein_proteus_sort.csv
T12_325_trypsin_PeaksDB_protein_proteus.txt
T12_332_undigested_PeaksDB_protein_proteus
T2_323_trypsin_PeaksDB_protein_proteus
T2_323_trypsin_PeaksDB_protein_proteus_sort.csv
T2_323_trypsin_PeaksDB_protein_proteus.txt
T2_330_undigested_PeaksDB_protein_proteus
T2_330_undigested_PeaksDB_protein_proteus_sort.csv
T2_330_undigested_PeaksDB_protein_proteus.txt
T5_324_trypsin_PeaksDB_protein_proteus
T5_324_trypsin_PeaksDB_protein_proteus_sort.csv
T

In [4]:
!head T5_331_undigested_PeaksDB_protein_proteus


Skip to content
Using UW Mail with screen readers
Meet
Hangouts
1 of 16,525
Proteus 2.0 done
Inbox
proteus2@wishartlab.com
	


In [5]:
# remove empty lines
# remove empty spaces from lines
# remove carat protein sequence names
# collapse the 2 lines from every protein sequence line (protein seq, secondard stucture pred.) into one line

!sed '/^[[:space:]]*$/d' T5_331_undigested_PeaksDB_protein_proteus | cat \
| sed '/>/d' \
| tr -d "[:blank:]" > T5_331_undigested_PeaksDB_protein_proteus.txt

!awk '{printf "%s%s",$0,(NR%2?FS:RS)}' T5_331_undigested_PeaksDB_protein_proteus.txt > \
T5_331_undigested_PeaksDB_protein_proteus_sort.csv

In [6]:
!head T5_331_undigested_PeaksDB_protein_proteus_sort.csv

Skiptocontent UsingUWMailwithscreenreaders
Meet Hangouts
1of16,525 Proteus2.0done
Inbox proteus2@wishartlab.com
Mon,Apr5,12:23PM(1dayago) THANKYOUFORUSINGPROTEUS2************************************************************PROTEUS2isoperatedandmaintainedbytheGenomeCanadaBioinformatic
13 proteus2@wishartlab.com
5:48PM(1hourago) ******************************Numberofsequencessubmitted:29Numberofsequencessuccessfullyprocessed:29Timeofsubmission:06:09:02Apr06,2021Timeo
KMKLAILAALFGSAAAFAPSQTGKASTQLRAFEDELGAQPPLGFFDPFGM CCCCHHHHHHHHHHCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
LSGDCTQERFDRLRYVEIKHGRICMLAFLGQVVTRAGIHLPGSINYAGDS CCCCCCHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHCCCCCCCCCCC
FDSFPNGVAALFGPNSIPTAGLVQIIAFIGVLECAFMRDVPGTGNEFVGD CCCCCCCCCCCCCCCCCHHHHHHHHHHHHHHHHHHHHHCCCCCCCCCCCC


In [7]:
# read into pandas with space delimeter

PeaksDB_331_prot = pd.read_csv("T5_331_undigested_PeaksDB_protein_proteus_sort.csv", delim_whitespace=True, header=None)


# name columns

PeaksDB_331_prot.columns =['Stripped protein sequence', 'Secondary structure pred.'] 

In [8]:
PeaksDB_331_prot.head()

,Stripped protein sequence,Secondary structure pred.
0,Skiptocontent,UsingUWMailwithscreenreaders
1,Meet,Hangouts
2,"1of16,525",Proteus2.0done
3,Inbox,proteus2@wishartlab.com
4,"Mon,Apr5,12:23PM(1dayago)",THANKYOUFORUSINGPROTEUS2**********************...


In [9]:
# add a column with the stripped peptide length (number of AAs)
PeaksDB_331_prot['Sequence length'] = PeaksDB_331_prot['Stripped protein sequence'].apply(len)

# use a count function to enumerate the # of C's (coil residues) for each peptide
PeaksDB_331_prot['C'] = PeaksDB_331_prot['Secondary structure pred.'].str.count("C")

# use a count function to enumerate the # of H's (helices residues) in each peptide
PeaksDB_331_prot['H'] = PeaksDB_331_prot['Secondary structure pred.'].str.count("H")

# use a count function to enumerate the # of E's (beta strand residues) in each peptide
PeaksDB_331_prot['E'] = PeaksDB_331_prot['Secondary structure pred.'].str.count("E")

#use a count function to enumerate the # of T's (membrane helix residues) in each peptide
PeaksDB_331_prot['T'] = PeaksDB_331_prot['Secondary structure pred.'].str.count("T")

# use a count function to enumerate the # of B's (membrane strand residues) in each peptide
PeaksDB_331_prot['B'] = PeaksDB_331_prot['Secondary structure pred.'].str.count("B")

# use a count function to enumerate the # of B's (signal peptide residues) in each peptide
PeaksDB_331_prot['S'] = PeaksDB_331_prot['Secondary structure pred.'].str.count("S")

# use a count function to enumerate the # of c's (cleavage site residues) in each peptide
PeaksDB_331_prot['c'] = PeaksDB_331_prot['Secondary structure pred.'].str.count("c")

# add a column with the % C
PeaksDB_331_prot['% C'] = PeaksDB_331_prot['C'] / PeaksDB_331_prot['Sequence length']

# add a column with the % H
PeaksDB_331_prot['% H'] = PeaksDB_331_prot['H'] / PeaksDB_331_prot['Sequence length']

# add a column with the % E
PeaksDB_331_prot['% E'] = PeaksDB_331_prot['E'] / PeaksDB_331_prot['Sequence length']

# add a column with the % T
PeaksDB_331_prot['% T'] = PeaksDB_331_prot['T'] / PeaksDB_331_prot['Sequence length']

# add a column with the % B
PeaksDB_331_prot['% B'] = PeaksDB_331_prot['B'] / PeaksDB_331_prot['Sequence length']

# add a column with the % S
PeaksDB_331_prot['% S'] = PeaksDB_331_prot['S'] / PeaksDB_331_prot['Sequence length']

# add a column with the % c
PeaksDB_331_prot['% c'] = PeaksDB_331_prot['c'] / PeaksDB_331_prot['Sequence length']

# additive check

PeaksDB_331_prot['% check'] = PeaksDB_331_prot['% C'] + PeaksDB_331_prot['% H'] + PeaksDB_331_prot['% E'] \
                                + PeaksDB_331_prot['% T'] + PeaksDB_331_prot['% B'] + \
                                PeaksDB_331_prot['% S'] + PeaksDB_331_prot['% c']

In [10]:
PeaksDB_331_prot.head()

,Stripped protein sequence,Secondary structure pred.,Sequence length,C,H,E,T,B,S,c,% C,% H,% E,% T,% B,% S,% c,% check
0,Skiptocontent,UsingUWMailwithscreenreaders,13,0,0,0,0,0,0,1,0.00,0.00,0.00,0.00,0.00,0.00,0.076923,0.076923
1,Meet,Hangouts,4,0,1,0,0,0,0,0,0.00,0.25,0.00,0.00,0.00,0.00,0.000000,0.250000
2,"1of16,525",Proteus2.0done,9,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000
3,Inbox,proteus2@wishartlab.com,5,0,0,0,0,0,0,1,0.00,0.00,0.00,0.00,0.00,0.00,0.200000,0.200000
4,"Mon,Apr5,12:23PM(1dayago)",THANKYOUFORUSINGPROTEUS2**********************...,25,1,1,2,3,1,3,1,0.04,0.04,0.08,0.12,0.04,0.12,0.040000,0.480000


In [11]:
index = ['331 total']

data = {
        '% C total': PeaksDB_331_prot['% C'].sum(),
        '% H total': PeaksDB_331_prot['% H'].sum(),
        '% E total': PeaksDB_331_prot['% E'].sum(),
        '% T total': PeaksDB_331_prot['% T'].sum(),
        '% B total': PeaksDB_331_prot['% B'].sum(),
        '% S total': PeaksDB_331_prot['% S'].sum(),
        '% c total': PeaksDB_331_prot['% c'].sum(),
        '% check sum': PeaksDB_331_prot['% check'].sum()
       }

PeaksDB_331_prot_totals = pd.DataFrame(data, columns=['% C total', '% H total', '% E total', '% T total', \
                                                      '% B total',  '% S total', '% c total', \
                                                      '% check sum'], index=index)

PeaksDB_331_prot_totals['overall % sum'] = PeaksDB_331_prot_totals['% C total'] \
                                            + PeaksDB_331_prot_totals['% H total'] \
                                            + PeaksDB_331_prot_totals['% E total'] \
                                            + PeaksDB_331_prot_totals['% T total'] \
                                            + PeaksDB_331_prot_totals['% B total'] \
                                            + PeaksDB_331_prot_totals['% S total'] \
                                            + PeaksDB_331_prot_totals['% c total'] 


PeaksDB_331_prot_totals['overall % C'] = PeaksDB_331_prot_totals['% C total'] / PeaksDB_331_prot_totals['overall % sum']

PeaksDB_331_prot_totals['overall % H'] = PeaksDB_331_prot_totals['% H total'] / PeaksDB_331_prot_totals['overall % sum']

PeaksDB_331_prot_totals['overall % E'] = PeaksDB_331_prot_totals['% E total'] / PeaksDB_331_prot_totals['overall % sum']

PeaksDB_331_prot_totals['overall % T'] = PeaksDB_331_prot_totals['% T total'] / PeaksDB_331_prot_totals['overall % sum']

PeaksDB_331_prot_totals['overall % B'] = PeaksDB_331_prot_totals['% B total'] / PeaksDB_331_prot_totals['overall % sum']

PeaksDB_331_prot_totals['overall % S'] = PeaksDB_331_prot_totals['% S total'] / PeaksDB_331_prot_totals['overall % sum']

PeaksDB_331_prot_totals['overall % c'] = PeaksDB_331_prot_totals['% c total'] / PeaksDB_331_prot_totals['overall % sum']

# write to csv

PeaksDB_331_prot_totals.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/analyses/proteus2/Proteins-Proteus2/PeaksDB_331_prot_undigested_totals")

PeaksDB_331_prot_totals.head()

,% C total,% H total,% E total,% T total,% B total,% S total,% c total,% check sum,overall % sum,overall % C,overall % H,overall % E,overall % T,overall % B,overall % S,overall % c
331 total,147.460209,80.845948,46.137316,30.211526,0.04,0.12,1.129423,305.944423,305.944423,0.481984,0.26425,0.150803,0.098748,0.000131,0.000392,0.003692
